In [ ]:
from roboflow import Roboflow
import os, glob

def download_dataset(api_key: str, workspace: str, project: str, version: int):
    rf = Roboflow(api_key=api_key)
    dataset = rf.workspace(workspace).project(project).version(version).download("yolov8")
    return dataset

dataset = download_dataset(api_key="K0xg5GEEinqPgaqjKKzz", workspace="matyworkspace", project="damagedhealthytrafficsigns", version=9)

image_dir = os.path.join(dataset.location, "test", "images")
label_dir = os.path.join(dataset.location, "test", "labels")
image_paths = glob.glob(os.path.join(image_dir, "*.jpg"))

In [ ]:
from get_predictions import save_predictions_to_file

predictions_file = "predictions.json"
if not os.path.exists(predictions_file):
    save_predictions_to_file(image_paths, predictions_file)

In [ ]:
from process_predictions import process_all_predictions

processed_predictions_file = "none.json"
if not os.path.exists(processed_predictions_file):
    process_all_predictions(image_paths, predictions_file, label_dir, processed_predictions_file)

In [ ]:
import pandas as pd
import json
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
with open("none.json", "r", encoding="utf-8") as f:
    data = json.load(f)
df = pd.DataFrame(data)
classes = ['healthy', 'damaged', 'background']
y_true = df['actual']
y_pred = df['predicted_label']

In [ ]:
import numpy as np

cm = confusion_matrix(y_true, y_pred, labels=classes)
cm_df = pd.DataFrame(cm, index=classes, columns=classes)


display(cm_df)

#cm[2, 2] = 0  # Set background-background to nan

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Only use 'healthy' and 'damaged' classes
labels = ['healthy', 'damaged']
precision = precision_score(df['actual'], df['predicted_label'], average=None, labels=labels)
recall = recall_score(df['actual'], df['predicted_label'], average=None, labels=labels)
f1 = f1_score(df['actual'], df['predicted_label'], average=None, labels=labels)

for cls, p, r, f in zip(labels, precision, recall, f1):
    print(f"{cls}: Precision={p:.3f}, Recall={r:.3f}, F1={f:.3f}")

In [ ]:
from sklearn.metrics import average_precision_score

ious = np.arange(0.5, 1.0, 0.05)
aps = []

for iou_thr in ious:
    # Filter detections by IoU threshold
    df_iou = df.copy()

    # AP for healthy
    y_true_healthy = (df_iou['actual'] == 'healthy').astype(int)
    y_score_healthy = df_iou['confidence'] * df_iou['score'] * (df_iou['iou'] >= iou_thr).astype(float) * (df_iou['score'] >= 0.5).astype(float)
    ap_healthy = average_precision_score(y_true_healthy, y_score_healthy) if len(y_true_healthy) > 0 and y_true_healthy.sum() > 0 else np.nan

    # AP for damaged
    y_true_damaged = (df_iou['actual'] == 'damaged').astype(int)
    y_score_damaged = df_iou['confidence'] * (1 - df_iou['score']) * (df_iou['iou'] >= iou_thr).astype(float) * (df_iou['score'] <= 0.5).astype(float)
    ap_damaged = average_precision_score(y_true_damaged, y_score_damaged) if len(y_true_damaged) > 0 and y_true_damaged.sum() > 0 else np.nan

    aps.append({'iou': iou_thr, 'AP_healthy': ap_healthy, 'AP_damaged': ap_damaged})

# Convert to DataFrame for display
aps_df = pd.DataFrame(aps)
aps_df['mAP'] = aps_df[['AP_healthy', 'AP_damaged']].mean(axis=1)
display(aps_df)

plt.figure(figsize=(8,5))
plt.plot(aps_df['iou'], aps_df['AP_healthy'], label='AP Healthy', marker='o', color='green')
plt.plot(aps_df['iou'], aps_df['AP_damaged'], label='AP Damaged', marker='o', color='red')
plt.plot(aps_df['iou'], aps_df['mAP'], label='mAP', marker='o', color='blue')
plt.xlabel('IoU Threshold')
plt.ylabel('Average Precision (AP)')
plt.title('AP and mAP vs IoU Threshold')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Calcular mAP@50:95 (media de mAP para IoU de 0.50 a 0.95 con paso 0.05)
map_50_95 = aps_df['mAP'].mean()
print(f"mAP@50:95 = {map_50_95:.3f}")